In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import time
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv('../Dataset/skincare_products.csv', delimiter = ',')
df.shape

(1138, 5)

In [10]:
df.head()

,product_name,product_url,product_type,ingredients,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"Aqua (Water), Caprylic/Capric Triglyceride, Ce...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"Aqua/Water, Homosalate, Glycerin, Octocrylene,...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"Aqua (Water), Sodium Hyaluronate, Sodium Hyalu...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"Aqua/Water/Eau, Ammonium Lactate, C12-15 Alkyl...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"Purified Water, Glycerin, Ceteareth-20 and Cet...",£16.00


In [4]:
df.describe()

,product_name,product_url,product_type,ingredients,price
count,1138,1138,1138,1138,1138
unique,1138,1126,14,1090,290
top,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/lancome-advanced...,Mask,"Aqua / Water, Bifida Ferment Lysate, Glycerin,...",£22.00
freq,1,4,124,4,33


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  1138 non-null   object
 1   product_url   1138 non-null   object
 2   product_type  1138 non-null   object
 3   ingredients   1138 non-null   object
 4   price         1138 non-null   object
dtypes: object(5)
memory usage: 44.6+ KB


In [11]:
ingredient_df = df['ingredients'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('ingredient')
ingredient_df = df.drop('ingredients', axis=1).join(ingredient_df)

In [12]:
ingredient_df.head()

,product_name,product_url,product_type,price,ingredient
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,£5.20,Aqua (Water)
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,£5.20,Caprylic/Capric Triglyceride
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,£5.20,Cetyl Alcohol
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,£5.20,Propanediol
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,£5.20,Stearyl Alcohol


Best for Redness
Key Ingredient = Aqua
Glycerin
Caprylic/Capric Triglyceride
Homosalate
Octocrylene


Sodium Hyaluronate: 1
Ammonium Lactate: 1
Purified Water: 1
Ceteareth-20: 1








---


In [9]:
df = df.drop('product_name')

KeyError: "['product_name'] not found in axis"

In [14]:
df2 = pd.read_csv('../Dataset/result.csv', delimiter = ',')
df2.shape

(1772, 12)

In [15]:
df2.head()

,label,url,brand,name,price,skin type,spf,concern,concern 2,concern 3,key ingredient,formulation
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,₹ 79,All,NaN,General Care,NaN,NaN,NaN,Cream
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,₹ 165,All,NaN,Uneven Skin Tone,Hydration,Dark Spots,Honey,Lotion
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,₹ 92,All,NaN,Dull Skin,Dryness,General Care,Aloe Vera,Cream
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,₹ 187,All,15 to 30,Softening and Smoothening,Softening and Smoothening,Sun Protection,Vitamin E,Cream
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Peach Milk Soft Creme 150g,₹ 192,All,NaN,General Care,NaN,NaN,NaN,Cream
